In [1]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict
from astropy.io import ascii
import os

In [2]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs

In [3]:
jupiter_table = ascii.read('../data/jupiter_table.dat')

scw_ids = jupiter_table['scw_id'].data
scw_versions = jupiter_table['scw_ver'].data
scw = [str(id).zfill(12) + ".00" + str(ver) for id, ver in zip(scw_ids, scw_versions)]

start = jupiter_table['start_date'].data
end = jupiter_table['end_date'].data
durations = [(e - s) for e,s in zip(end, start)]

jupiter_ra = jupiter_table['jupiter_ra'].data
jupiter_dec = jupiter_table['jupiter_dec'].data

## QUERYING 

In [4]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

15 - 30 keV

In [5]:
# Already processed ScWs

scw_to_match1 = []
directory1 = "../data/Jupiter/15-30keV/Images"
for filename in os.listdir(directory1):
    if filename.endswith(".fits"):
        scw_to_match1.append(filename[:16])

# scws different from the ones in the table
not_in_table = [s for s in scw_to_match1 if s not in scw]
print("Number of ScWs not in the table:", len(not_in_table))
print("Number of ScWs to query:", len(scw) - (len(scw_to_match1) - len(not_in_table)))

Number of ScWs not in the table: 0
Number of ScWs to query: 0


In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:
        
        # Check ScW isn't already in the FITS folder
        if scw_id in scw_to_match1:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "15",
            "E2_keV": "30",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

In [5]:
scw_to_match2 = []
directory2 = "../data/Jupiter/30-60keV/Images"
for filename in os.listdir(directory2):
    if filename.endswith(".fits"):
        scw_to_match2.append(filename[:16])

30 - 60 keV

In [6]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id in scw_to_match2:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "30",
            "E2_keV": "60",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700060010.001
Is complete  True
Trying SCW 026700070010.001
Is complete  False
Trying SCW 026700080010.001
Is complete  True
Trying SCW 026700090010.001
Is complete  True
Trying SCW 026700100010.001
Is complete  False
Trying SCW 026700110010.001
Is complete  True
Trying SCW 026700120010.001
Is complete  True
Trying SCW 026700130010.001
Is complete  True
Trying SCW 026700140010.001
Is complete  True
Trying SCW 026700150010.001
Is complete  True
Trying SCW 026700160010.001
Is complete  True
Trying SCW 026700170010.001
Is complete  False
Trying SCW 026700180010.001
Is complete  True
Trying SCW 026700190010.001
Is complete  True
Trying SCW 026700200010.001
Is complete  True
Trying SCW 026700210010.001
Is complete  True
Trying SCW 026700230010.001
Is complete  True
Trying SCW 026700240010.001
Is complete  True
Trying SCW 026700250010.001
Is complete  True
Trying SCW 026700260010.001
Is complete  True
Trying SCW 026700270010.001
Is complete  True
Trying SCW 026700280010.001
Is 

60 - 100 keV

In [5]:
# Already processed ScWs

scw_to_match3 = []
directory3 = "../data/Jupiter/60-100keV/Images"
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

# scws different from the ones in the table
not_in_table = [s for s in scw_to_match3 if s not in scw]
print("Number of ScWs not in the table:", len(not_in_table))
print("Number of ScWs to query:", len(scw) - (len(scw_to_match3) - len(not_in_table)))

Number of ScWs not in the table: 0
Number of ScWs to query: 239


In [6]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id in scw_to_match3:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "60",
            "E2_keV": "100",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")


    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700170010.001


query failed!
Remote server message:-> Error when getting query products
Remote server error_message-> Instrument: isgri, product: isgri_image

The support team has been notified, and we are investigating to resolve the issue as soon as possible

If you are willing to help us, please use the "Write a feedback" button below. We will make sure to respond to any feedback provided
Remote server debug_message-> DDAException('AnalysisException:ProduceDisabledException("not allowed to produce but has to! at [mosaic_ii_skyimage.v2.2.4(completescw:None);Virtual;Complete139912847781024]; hashe: (\'analysis\', (\'list\', (\'analysis\', None, \'ImageBins.onebin_60.0_100.0.one_bin_60_100\'), (\'analysis\', (\'analysis\', None, \'ICRoot.v1.default-isdc\'), \'IBIS_ICRoot.v0\'), (\'analysis\', (\'list\', \'processing_definition:90d87759\', (\'analysis\', None, \'IDScWList.v0_one_026700170010.001\')), \'ImageGroups.v1.2\'), (\'analysis\', \'500s2x88\', \'MosaicImagingConfig.v0\'), (\'analysis\', (\'lis

Query failed for SCW 026700170010.001: RemoteException (line 99): Error when getting query products
Trying SCW 214500460010.001
Is complete  True
Trying SCW 214500490010.001
Is complete  True
Trying SCW 214500500010.001
Is complete  True
Trying SCW 214500510010.001
Is complete  True
Trying SCW 214600260010.001
Is complete  True
Trying SCW 214600270010.001
Is complete  True
Trying SCW 214600280010.001
Is complete  True
Trying SCW 214600330010.001
Is complete  True
Trying SCW 214600340010.001
Is complete  True
Trying SCW 214600350010.001
Is complete  True
Trying SCW 214600360010.001
Is complete  True
Trying SCW 214600370010.001
Is complete  True
Trying SCW 214600410010.001
Is complete  True
Trying SCW 214600420010.001
Is complete  True
Trying SCW 214600430010.001
Is complete  True
Trying SCW 214600440010.001
Is complete  True
Trying SCW 214600450010.001
Is complete  True
Trying SCW 214600520010.001
Is complete  True
Trying SCW 214600530010.001
Is complete  True
Trying SCW 214600540010.00

possibly temporary problem in calling server: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.astro.unige.ch', port=443): Read timed out. (read timeout=120)")) in 962.2 seconds, 199 tries left, sleeping 10 seconds until retry


Trying SCW 215100200010.001
Is complete  True
Trying SCW 215100210010.001
Is complete  True
Trying SCW 215100220010.001
Is complete  True
Trying SCW 215100230010.001
Is complete  True
Trying SCW 215100240010.001
Is complete  True
Trying SCW 215100250010.001
Is complete  True
Trying SCW 215100260010.001
Is complete  True
Trying SCW 215100280010.001
Is complete  True
Trying SCW 215100290010.001
Is complete  True
Trying SCW 215100300010.001
Is complete  True
Trying SCW 215100310010.001
Is complete  True
Trying SCW 215100320010.001
Is complete  True
Trying SCW 215100330010.001
Is complete  True
Trying SCW 215100340010.001
Is complete  True
Trying SCW 215100350010.001
Is complete  True
Trying SCW 215100360010.001
Is complete  True
Trying SCW 215100400010.001
Is complete  True
Trying SCW 215100410010.001
Is complete  True
Trying SCW 215100510010.001
Is complete  True
Trying SCW 215100520010.001
Is complete  True
Trying SCW 215100530010.001
Is complete  True
Trying SCW 215100560010.001
Is com

100 - 200 keV

In [7]:
# Already processed ScWs

scw_to_match4 = []
directory4 = "../data/Jupiter/100-200keV/Images"
for filename in os.listdir(directory4):
    if filename.endswith(".fits"):
        scw_to_match4.append(filename[:16])

# scws different from the ones in the table
not_in_table = [s for s in scw_to_match4 if s not in scw]
print("Number of ScWs not in the table:", len(not_in_table))
print("Number of ScWs to query:", len(scw) - (len(scw_to_match4) - len(not_in_table)))

Number of ScWs not in the table: 0
Number of ScWs to query: 915


In [ ]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id in scw_to_match4:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "100",
            "E2_keV": "200",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700060010.001
Is complete  False
Trying SCW 026700070010.001
Is complete  False
Trying SCW 026700080010.001
Is complete  False
Trying SCW 026700090010.001
Is complete  False
Trying SCW 026700100010.001
Is complete  False
Trying SCW 026700110010.001
Is complete  False
Trying SCW 026700120010.001
Is complete  False
Trying SCW 026700130010.001
Is complete  False
Trying SCW 026700140010.001
Is complete  False
Trying SCW 026700150010.001
Is complete  False
Trying SCW 026700160010.001
Is complete  False
Trying SCW 026700170010.001
Is complete  False
Trying SCW 026700180010.001
Is complete  False
Trying SCW 026700190010.001
Is complete  False
Trying SCW 026700200010.001
Is complete  False
Trying SCW 026700210010.001
Is complete  False
Trying SCW 026700230010.001
Is complete  False
Trying SCW 026700240010.001
Is complete  False
Trying SCW 026700250010.001
Is complete  False
Trying SCW 026700260010.001
Is complete  False
Trying SCW 026700270010.001
Is complete  False
Trying SCW 02

possibly temporary problem in calling server: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='www.astro.unige.ch', port=443): Read timed out. (read timeout=120)")) in 3365.9 seconds, 199 tries left, sleeping 10 seconds until retry


Is complete  True
Trying SCW 214600550010.001
Is complete  False
Trying SCW 214900260010.001
Is complete  False
Trying SCW 214900350010.001
Is complete  False
Trying SCW 214900360010.001
Is complete  False
Trying SCW 214900370010.001
Is complete  False
Trying SCW 214900380010.001
Is complete  False
Trying SCW 214900390010.001
Is complete  False
Trying SCW 214900400010.001
Is complete  False
Trying SCW 214900410010.001
Is complete  False
Trying SCW 214900420010.001
Is complete  False
Trying SCW 214900520010.001
Is complete  False
Trying SCW 214900530010.001
Is complete  False
Trying SCW 214900540010.001
Is complete  False
Trying SCW 214900550010.001
Is complete  False
Trying SCW 214900560010.001
Is complete  False
Trying SCW 214900570010.001
Is complete  False
Trying SCW 214900580010.001
Is complete  False
Trying SCW 214900680010.001
Is complete  False
Trying SCW 214900690010.001
Is complete  False
Trying SCW 214900700010.001
Is complete  False
Trying SCW 214900710010.001
Is complete  F

3 - 15 keV

In [ ]:
scw_to_match3 = []
directory3 = "../data/Jupiter/3-15keV/Images"
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

In [ ]:
jemx_validity = []
for s in scw:
    for row in jupiter_table:
        if str(row['scw_id']).zfill(12) == s[:12]:
            jemx_validity.append(row['jemx_valid'])

In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for i, (scw_id, start_time, end_time, jra, jdec) in enumerate(filtered_scws):

        if scw_id in scw_to_match3:
            continue

        if jemx_validity[i] == 0:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "3",
            "E2_keV": "15",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "jemx",
            "jemx_num": '2',
            "osa_version": "OSA11.2",
            "product": "jemx_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Save FITS

In [7]:
new_results = []
new_scws = []
for scw_id, data in data_by_date.items():
    if data is not None:
        new_results.append(data)
        new_scws.append(scw_id)

In [9]:
# Save FITS files
import os

output_dir = "../data/Jupiter/60-100keV/Images/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Check if the directory is empty (optional, comment out if querying additional ScWs & see above those missing through scw_to_match)
#if os.path.exists(output_dir) and os.listdir(output_dir):
    #print(f"Directory '{output_dir}' is not empty. Aborting to prevent overwriting.")
else:
    for i, data in enumerate(new_results):
    #if new_scws[i] in scw_to_match:
        im = OdaImage(data)
        im.write_fits(os.path.join(output_dir, f"{new_scws[i]}"))